In [ ]:
import sys
from datetime import datetime, timezone
import json
import json
import serial
import requests

### ⚒  How to collect data (connect sensor with arduino)

Hereinafter we are going to set up a very simple system to monitor the temperature and the humidity by using **Arduino** and the **DHT11**.

* [Arduino Nano](https://store.arduino.cc/arduino-nano)
* [DHT11](https://components101.com/dht11-temperature-sensor)

1. Connecting the **DHT11** to the **Arduino Nano**

| **DHT11**             |  __  | **Arduino** |
| --------------------- | ---- | ----------- |
| **GND** or **-**      | <--> | **GND**     |
| **VCC** or **+**      | <--> | **5V**      |
| **Data**              | <--> | **D2**      |

### ⚒  Connect the **Arduino Nano** with the **Computer**

**In Linux OS**:<br>
Run the following command in a bash shell to get the port name and to set reading priviledges<br>

$ dmesg | tail

$ sudo chmod 666 /dev/ttyACM0

**In Windows OS**:<br>


### ⚒ Set the communication configuration

In [ ]:
conf = {
    "port": "/dev/ttyUSB0", # system port where the arduino is connected
    "baudrate": 9600, # velocity of the port
    "url": "http://istsos.org/istsos", # web address of the istSOS server
    "service": "maxdemo", # service name of the istSOS service
    "procedure_id": None, # ID of the procedure already registered into istSOS
    "auth": None # Basic Authentication tuple: ("admin", "batman") set to None if no auth required
}

### ⚒  Create a new procedure in istSOS to host data using the WAlib

In [ ]:
sensor = {
    "system_id":"ARDUINO_TEST",
    "system":"ARDUINO_TEST",
    "description":"Meteo Station in Bellinzona",
    "keywords": "weather, meteorological, IST",
    "identification":[
        {
            "name":"uniqueID",
            "definition":"urn:ogc:def:identifier:OGC:uniqueID",
            "value":"urn:ogc:def:procedure:x-istsos:1.0:BELLINZONA"
        }
    ],
    "classification":[
        {
            "name":"System Type",
            "definition":"urn:ogc:def:classifier:x-istsos:1.0:systemType",
            "value":"insitu-fixed-point"
        },
        {
            "name":"Sensor Type",
            "definition":"urn:ogc:def:classifier:x-istsos:1.0:sensorType",
            "value":"Davis weather station"
        }
    ],
    "characteristics":"",
    "contacts":[],
    "documentation":[],
    "capabilities":[],
    "location":{
        "type":"Feature",
        "geometry":{
            "type":"Point",
            "coordinates":["9.01976","46.20322","226.3"]
        },
        "crs":{
            "type":"name",
            "properties":{"name":"4326"}
        },
        "properties":{
            "name":"ARDUINO_TEST"
        }
    },
    "interfaces":"",
    "inputs":[],
    "outputs":[
        {
            "name":"Time",
            "definition":"urn:ogc:def:parameter:x-istsos:1.0:time:iso8601",
            "uom":"iso8601",
            "description":"",
            "constraint":{}
        },
        {
            "name":"air-humidity-relative",
            "definition":"urn:ogc:def:parameter:x-istsos:1.0:meteo:air:humidity:relative",
            "uom":"%",
            "description":"-",
            "constraint":{
                "role":"urn:ogc:def:classifiers:x-istsos:1.0:qualityIndex:check:reasonable",
                "interval":["0","100"]
            }
        },
        {
            "name":"air-temperature",
            "definition":"urn:ogc:def:parameter:x-istsos:1.0:meteo:air:temperature",
            "uom":"°C",
            "description":"conversion from resistance to temperature",
            "constraint":{
                "role":"urn:ogc:def:classifiers:x-istsos:1.0:qualityIndex:check:reasonable",
                "interval":["-40","60"]
            }
        }
    ],
    "history":[]
}

### ⚒ Register sensor and read sensor serial ID

In [ ]:
url = conf['url'] + '/wa/istsos/services/' + conf['service'] + '/procedures'

res = requests.post(url, json.dumps(sensor), auth=conf['auth'])
if res.status_code == 200:
    conf['procedure_id'] = res.json()['message'].split('urn:ogc:def:sensor:x-istsos:1.0:')[1][:32]
else:
    print(res.json())
print(conf)

### ⚒ Reading data from serial port
The following part of the course is useful to understand how the data can be read from a serial port

In [ ]:
max_count = 4
count = 0
with serial.Serial(conf['port'], 9600, timeout=10) as ser:
    while count < max_count:
        line = ser.readline()   # read a '\n' terminated line
        print("Line in Bytes String --> {}".format(
            line
        ))
        print("Line decoded         --> {}".format(
            line.decode()
        ))
        count += 1
print('Stopped')

### ⚒ Sending data to istSOS

In [ ]:
max_count = 60
count = 0
post_url = "{}/wa/istsos/services/{}/operations/fastinsert".format(
    conf['url'],
    conf['service']
)
with serial.Serial(conf['port'], 9600, timeout=10) as ser:
    while count < max_count:
        line = ser.readline()   # read a '\n' terminated line
        if count > 0:
            values = line.decode().strip('\r\n')
            data = "{};{},{}".format(
                conf['procedure_id'],
                datetime.now(timezone.utc).isoformat(),
                values
            )
            req = requests.post(
                post_url,
                data,
                auth=conf['auth']
            )
            if req.status_code == 200:
                print(data)
                print(req.text)
            else:
                print(data)
                print('ERROR: Data not sent')
            
        count += 1
print('Stopped')